In [31]:
import tensorflow as tf
import tensorflow.keras.datasets as tfds
from utils.layer_units import *
from utils.ResAttNet import ResAttNet
import pydotplus
import numpy as np


# Plot configurations
%matplotlib inline

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
(X_train,y_train),(X_test,y_test) = tfds.cifar100.load_data()

In [33]:
X_valid = X_train[49000:]
y_valid = y_train[49000:]
X_train = X_train[:49000]
y_train = y_train[:49000]

In [34]:
# center but not scale data
# we found scale not very stable
def preprocess(X):
    # pixel wise center
    ret = X.astype(np.float32) - np.mean(X,axis=0)
    return ret

In [35]:
X_train = preprocess(X_train)
X_valid = preprocess(X_valid)
X_test = preprocess(X_test)

In [36]:
my_RANet = ResAttNet(100,256)

In [37]:
ipt = tf.keras.Input(shape=(32,32,3))
model_out = my_RANet.build(ipt, pre_conv = True, pre_pooling = False, 
             attention_num = 3,network_param = [1,1,1],
             resid_params = [[64,3,1,0],[64,3,1,0],[64,1,1,0]],
             skip_param = [True, 1])
model = tf.keras.Model(ipt,model_out)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_351 (Conv2D)             (None, 32, 32, 64)   1792        input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_249 (BatchN (None, 32, 32, 64)   256         conv2d_351[0][0]                 
__________________________________________________________________________________________________
activation_257 (Activation)     (None, 32, 32, 64)   0           batch_normalization_249[0][0]    
____________________________________________________________________________________________

In [38]:
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9,nesterov=True)

model.compile(optimizer = 'Adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['sparse_categorical_accuracy'],
)

In [39]:
# ImageDataGenerator code referred from hw2 Task4
# and referred from https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
datagen = tf.keras.preprocessing.image.ImageDataGenerator(shear_range=0.1,zoom_range=0.2,horizontal_flip=True,rotation_range=30,width_shift_range=0.1,
                             height_shift_range=0.1)
datagen.fit(X_train)

In [ ]:
#history = model.fit(X_train,y_train,epochs=10,batch_size=128,validation_split=0.2,callbacks=[tensorboard_callback])
bs = 128
history = model.fit(datagen.flow(X_train,y_train,batch_size=bs),epochs=30,steps_per_epoch=len(X_train)//bs,
                    validation_data = (X_valid,y_valid))

Epoch 1/30
382/382 [==============================] - 245s 641ms/step - loss: 4.4589 - sparse_categorical_accuracy: 0.0909 - val_loss: 3.7137 - val_sparse_categorical_accuracy: 0.1320
Epoch 2/30
382/382 [==============================] - 244s 639ms/step - loss: 3.4339 - sparse_categorical_accuracy: 0.1715 - val_loss: 3.5279 - val_sparse_categorical_accuracy: 0.1700
Epoch 3/30
 95/382 [======>.......................] - ETA: 3:00 - loss: 3.2046 - sparse_categorical_accuracy: 0.2109

In [116]:
model.save('cifar_100_two_attention_64_3_resid.h5')

In [117]:
model.evaluate(x=X_test,y=y_test)

313/313 [==============================] - 5s 14ms/step - loss: 2.1031 - sparse_categorical_accuracy: 0.4934


[2.103074312210083, 0.4934000074863434]

In [11]:
import pandas as pd
hist_normalized = pd.DataFrame(history.history)

In [12]:
hist_normalized.to_csv('hist_two_attention_triple_64_3_3_one_dense_no_prepooling.csv',sep=',')